<a href="https://colab.research.google.com/github/dikshant182004/Kaggle-Notebooks/blob/main/texual_data_using_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [78]:
import pandas as pd
df= pd.read_csv('/content/100_Unique_QA_Dataset.csv')
df.head()

,question,answer
0,What is the capital of France?,Paris
1,What is the capital of Germany?,Berlin
2,Who wrote 'To Kill a Mockingbird'?,Harper-Lee
3,What is the largest planet in our solar system?,Jupiter
4,What is the boiling point of water in Celsius?,100


In [79]:
def tokenize(text):
  text = text.lower()
  text = text.replace('?','')
  text = text.replace("'","")
  return text.split()

In [80]:
tokenize('What is the capital of France?')

['what', 'is', 'the', 'capital', 'of', 'france']

In [81]:
vocab = {'<UNK>':0}

In [82]:
def build_vocab(row):
  tokenized_qn = tokenize(row['question'])
  tokenized_ans= tokenize(row['answer'])

  merged_tokens = tokenized_qn + tokenized_ans

  for token in merged_tokens:
    if token not in vocab:
      vocab[token] = len(vocab)


In [83]:
df.apply(build_vocab, axis=1)

,0
0,None
1,None
2,None
3,None
4,None
...,...
85,None
86,None
87,None
88,None


In [84]:
vocab['what']

1

In [85]:
len(vocab)

324

In [86]:
# convert words to numerical indices

def text_to_indices(text,vocab):

  indexed_text=[]

  for token in tokenize(text):

    if token in vocab:
      indexed_text.append(vocab[token])
    else:
      indexed_text.append(vocab['<UNK>'])

  return indexed_text

In [87]:
text_to_indices("What is campusx", vocab)

[1, 2, 0]

In [88]:
import torch
from torch.utils.data import Dataset, DataLoader

In [89]:
class QAdataset(Dataset):

  def __init__(self,df,vocab):
    self.df =df
    self.vocab = vocab

  def __len__(self):
    return self.df.shape[0]

  def __getitem__(self,index):
    numerical_qn = text_to_indices(self.df.iloc[index]['question'],self.vocab)
    numerical_answer = text_to_indices(self.df.iloc[index]['answer'],self.vocab)

    return torch.tensor(numerical_qn), torch.tensor(numerical_answer)


In [90]:
dataset =QAdataset(df, vocab)

In [91]:
dataloader = DataLoader(dataset,batch_size=1, shuffle=True)

In [92]:
for qn, answer in dataloader:
  print(qn,answer[0])

tensor([[  1,   2,   3,   4,   5, 113]]) tensor([114])
tensor([[ 42,   2,   3, 210, 137, 168, 211, 169]]) tensor([113])
tensor([[ 1,  2,  3,  4,  5, 73]]) tensor([74])
tensor([[42, 18,  2, 62, 63,  3, 64, 18]]) tensor([65])
tensor([[ 78,  79, 129,  81,  19,   3,  21,  22]]) tensor([36])
tensor([[ 42, 125,   2,  62,  63,   3, 126, 127]]) tensor([128])
tensor([[ 78,  79, 150, 151,  14, 152, 153]]) tensor([154])
tensor([[ 10,  96,   3, 104, 239]]) tensor([240])
tensor([[ 1,  2,  3, 59, 25,  5, 26, 19, 60]]) tensor([61])
tensor([[ 42, 174,   2,  62,  39, 175, 176,  12, 177, 178]]) tensor([179])
tensor([[  1,   2,   3, 212,   5,  14, 213, 214]]) tensor([215])
tensor([[10,  2,  3, 66,  5, 67]]) tensor([68])
tensor([[ 10,   2,  62,  63,   3, 283,   5, 284]]) tensor([285])
tensor([[10, 55,  3, 56,  5, 57]]) tensor([58])
tensor([[ 42, 137,   2, 226,  12,   3, 227, 228]]) tensor([155])
tensor([[ 42, 107,   2, 108,  19, 109]]) tensor([110])
tensor([[1, 2, 3, 4, 5, 8]]) tensor([9])
tensor([[  1,  

no padding is req as we are sending one row at a time

In [93]:
import torch.nn as nn

In [94]:
class SimpleRNN(nn.Module):

  def __init__(self,vocab_size):
    # in seqn models we get hidden outputs too thatswhy we dont use the sequential block here
    super().__init__()
    self.embedding=nn.Embedding(vocab_size,embedding_dim=50)
    self.rnn= nn.RNN(50,64, batch_first=True)
    self.fc= nn.Linear(64,vocab_size)

  def forward(self,question):
    embedding_qn=self.embedding(question)
    hidden,final= self.rnn(embedding_qn)  # it gives hidden outputs too
    output= self.fc(final.squeeze(0))

    return output

In [95]:
x = nn.Embedding(324, embedding_dim=50)
y = nn.RNN(50, 64, batch_first=True)
z = nn.Linear(64, 324)

a = dataset[0][0].reshape(1,6)
print("shape of a:", a.shape)
b = x(a)
print("shape of b:", b.shape)
c, d = y(b)
print("shape of c:", c.shape)
print("shape of d:", d.shape)

e = z(d.squeeze(0))

print("shape of e:", e.shape)

shape of a: torch.Size([1, 6])
shape of b: torch.Size([1, 6, 50])
shape of c: torch.Size([1, 6, 64])
shape of d: torch.Size([1, 1, 64])
shape of e: torch.Size([1, 324])


In [106]:
learning_rate = 0.001
epochs = 20

In [97]:
model = SimpleRNN(len(vocab))

In [98]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [107]:
for epoch in range(epochs):
  total_loss=0
  for qn,ans in dataloader:

    optimizer.zero_grad()
    output= model(qn)
    loss= criterion(output,answer[0])

    loss.backward()
    optimizer.step()

    total_loss = total_loss + loss.item()

  print(f"Epoch : {epoch+1}, Loss:{total_loss:4f}")

Epoch : 1, Loss:0.141327
Epoch : 2, Loss:0.121443
Epoch : 3, Loss:0.105524
Epoch : 4, Loss:0.092553
Epoch : 5, Loss:0.081847
Epoch : 6, Loss:0.072864
Epoch : 7, Loss:0.065265
Epoch : 8, Loss:0.058770
Epoch : 9, Loss:0.053166
Epoch : 10, Loss:0.048298
Epoch : 11, Loss:0.044042
Epoch : 12, Loss:0.040297
Epoch : 13, Loss:0.036982
Epoch : 14, Loss:0.034033
Epoch : 15, Loss:0.031395
Epoch : 16, Loss:0.029032
Epoch : 17, Loss:0.026898
Epoch : 18, Loss:0.024971
Epoch : 19, Loss:0.023227
Epoch : 20, Loss:0.021640


In [108]:
def predict(model,qn, threshold=0.5):

  numerical_qn= text_to_indices(qn,vocab)
  # converting to tensor as model expects tensor only
  qn_tensor = torch.tensor(numerical_qn).unsqueeze(0)
  output= model(qn_tensor)

  probs= torch.nn.functional.softmax(output,dim=1) # extracting probs
  value,index = torch.max(probs, dim=1)
  print(value,index)

  if value < threshold:
    print("I don't know")

  print(list(vocab.keys())[index])

In [109]:
predict(model, "What is the capital of london?")

tensor([0.9998], grad_fn=<MaxBackward0>) tensor([185])
vaticancity


In [105]:
list(vocab.keys())[316]

'batman'

It is not predicting correct output as there is some issue in data it is processing

## Now lets work with the LSTM for our sequential data

In [110]:
!pip install nltk

In [159]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from collections import Counter
from torch.utils.data import Dataset, DataLoader
from nltk.tokenize import word_tokenize
import nltk

In [160]:
document = """About the Program
What is the course fee for  Data Science Mentorship Program (DSMP 2023)
The course follows a monthly subscription model where you have to make monthly payments of Rs 799/month.
What is the total duration of the course?
The total duration of the course is 7 months. So the total course fee becomes 799*7 = Rs 5600(approx.)
What is the syllabus of the mentorship program?
We will be covering the following modules:
Python Fundamentals
Python libraries for Data Science
Data Analysis
SQL for Data Science
Maths for Machine Learning
ML Algorithms
Practical ML
MLOPs
Case studies
You can check the detailed syllabus here - https://learnwith.campusx.in/courses/CampusX-Data-Science-Mentorship-Program-637339afe4b0615a1bbed390
Will Deep Learning and NLP be a part of this program?
No, NLP and Deep Learning both are not a part of this program’s curriculum.
What if I miss a live session? Will I get a recording of the session?
Yes all our sessions are recorded, so even if you miss a session you can go back and watch the recording.
Where can I find the class schedule?
Checkout this google sheet to see month by month time table of the course - https://docs.google.com/spreadsheets/d/16OoTax_A6ORAeCg4emgexhqqPv3noQPYKU7RJ6ArOzk/edit?usp=sharing.
What is the time duration of all the live sessions?
Roughly, all the sessions last 2 hours.
What is the language spoken by the instructor during the sessions?
Hinglish
How will I be informed about the upcoming class?
You will get a mail from our side before every paid session once you become a paid user.
Can I do this course if I am from a non-tech background?
Yes, absolutely.
I am late, can I join the program in the middle?
Absolutely, you can join the program anytime.
If I join/pay in the middle, will I be able to see all the past lectures?
Yes, once you make the payment you will be able to see all the past content in your dashboard.
Where do I have to submit the task?
You don’t have to submit the task. We will provide you with the solutions, you have to self evaluate the task yourself.
Will we do case studies in the program?
Yes.
Where can we contact you?
You can mail us at nitish.campusx@gmail.com
Payment/Registration related questions
Where do we have to make our payments? Your YouTube channel or website?
You have to make all your monthly payments on our website. Here is the link for our website - https://learnwith.campusx.in/
Can we pay the entire amount of Rs 5600 all at once?
Unfortunately no, the program follows a monthly subscription model.
What is the validity of monthly subscription? Suppose if I pay on 15th Jan, then do I have to pay again on 1st Feb or 15th Feb
15th Feb. The validity period is 30 days from the day you make the payment. So essentially you can join anytime you don’t have to wait for a month to end.
What if I don’t like the course after making the payment. What is the refund policy?
You get a 7 days refund period from the day you have made the payment.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmail.com
Post registration queries
Till when can I view the paid videos on the website?
This one is tricky, so read carefully. You can watch the videos till your subscription is valid. Suppose you have purchased subscription on 21st Jan, you will be able to watch all the past paid sessions in the period of 21st Jan to 20th Feb. But after 21st Feb you will have to purchase the subscription again.
But once the course is over and you have paid us Rs 5600(or 7 installments of Rs 799) you will be able to watch the paid sessions till Aug 2024.
Why lifetime validity is not provided?
Because of the low course fee.
Where can I reach out in case of a doubt after the session?
You will have to fill a google form provided in your dashboard and our team will contact you for a 1 on 1 doubt clearance session
If I join the program late, can I still ask past week doubts?
Yes, just select past week doubt in the doubt clearance google form.
I am living outside India and I am not able to make the payment on the website, what should I do?
You have to contact us by sending a mail at nitish.campusx@gmai.com
Certificate and Placement Assistance related queries
What is the criteria to get the certificate?
There are 2 criterias:
You have to pay the entire fee of Rs 5600
You have to attempt all the course assessments.
I am joining late. How can I pay payment of the earlier months?
You will get a link to pay fee of earlier months in your dashboard once you pay for the current month.
I have read that Placement assistance is a part of this program. What comes under Placement assistance?
This is to clarify that Placement assistance does not mean Placement guarantee. So we dont guarantee you any jobs or for that matter even interview calls. So if you are planning to join this course just for placements, I am afraid you will be disappointed. Here is what comes under placement assistance
Portfolio Building sessions
Soft skill sessions
Sessions with industry mentors
Discussion on Job hunting strategies
"""


In [161]:
# tokenization
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [162]:
# tokenize
tokens = word_tokenize(document.lower())

In [163]:
vocab= {'<unk>':0}

for token in Counter(tokens).keys():
  if token not in vocab:
    vocab[token]=len(vocab)

vocab

{'<unk>': 0,
 'about': 1,
 'the': 2,
 'program': 3,
 'what': 4,
 'is': 5,
 'course': 6,
 'fee': 7,
 'for': 8,
 'data': 9,
 'science': 10,
 'mentorship': 11,
 '(': 12,
 'dsmp': 13,
 '2023': 14,
 ')': 15,
 'follows': 16,
 'a': 17,
 'monthly': 18,
 'subscription': 19,
 'model': 20,
 'where': 21,
 'you': 22,
 'have': 23,
 'to': 24,
 'make': 25,
 'payments': 26,
 'of': 27,
 'rs': 28,
 '799/month': 29,
 '.': 30,
 'total': 31,
 'duration': 32,
 '?': 33,
 '7': 34,
 'months': 35,
 'so': 36,
 'becomes': 37,
 '799': 38,
 '*': 39,
 '=': 40,
 '5600': 41,
 'approx': 42,
 'syllabus': 43,
 'we': 44,
 'will': 45,
 'be': 46,
 'covering': 47,
 'following': 48,
 'modules': 49,
 ':': 50,
 'python': 51,
 'fundamentals': 52,
 'libraries': 53,
 'analysis': 54,
 'sql': 55,
 'maths': 56,
 'machine': 57,
 'learning': 58,
 'ml': 59,
 'algorithms': 60,
 'practical': 61,
 'mlops': 62,
 'case': 63,
 'studies': 64,
 'can': 65,
 'check': 66,
 'detailed': 67,
 'here': 68,
 '-': 69,
 'https': 70,
 '//learnwith.campusx.i

In [164]:
len(vocab)

289

In [165]:
input_sentences = document.split('\n')

In [166]:
def text_to_indices(sentence, vocab):

  numerical_sentence = []

  for token in sentence:
    if token in vocab:
      numerical_sentence.append(vocab[token])
    else:
      numerical_sentence.append(vocab['<unk>'])

  return numerical_sentence


In [167]:
input_numerical_sentences = []

for sentence in input_sentences:
  input_numerical_sentences.append(text_to_indices(word_tokenize(sentence.lower()), vocab))


In [168]:
len(input_numerical_sentences)

78

In [169]:
# just converting the unlabelled raw data in supervized problem
training_sequence =[]

for sentence in input_numerical_sentences:
  for i in range(1,len(sentence)):

    training_sequence.append(sentence[:i+1])

In [170]:
training_sequence[:5]

[[1, 2], [1, 2, 3], [4, 5], [4, 5, 2], [4, 5, 2, 6]]

In [171]:
len_list =[]
# finding the maximum length training parameter

for sequence in training_sequence:
  len_list.append(len(sequence))

max(len_list)

62

In [172]:
padded_seq=[]
for sequence in training_sequence:

  padded_seq.append([0]*(max(len_list)-len(sequence))+sequence)

In [173]:
padded_training_sequence = torch.tensor(padded_seq, dtype=torch.long)

In [174]:
padded_training_sequence


tensor([[  0,   0,   0,  ...,   0,   1,   2],
        [  0,   0,   0,  ...,   1,   2,   3],
        [  0,   0,   0,  ...,   0,   4,   5],
        ...,
        [  0,   0,   0,  ..., 285, 176, 286],
        [  0,   0,   0,  ..., 176, 286, 287],
        [  0,   0,   0,  ..., 286, 287, 288]])

In [175]:
X = padded_training_sequence[:, :-1]
y = padded_training_sequence[:,-1]

In [176]:
class CustomDataset(Dataset):

  def __init__(self,X,y):
    self.X=X
    self.y=y

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self,idx):
    return self.X[idx], self.y[idx]

In [177]:
dataset = CustomDataset(X,y)

In [178]:
len(dataset)

942

In [179]:
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [180]:
class LSTMModel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
# not using the sequential model becz of the same reason of rnn

    self.embedding= nn.Embedding(vocab_size,100)
    self.lstm= nn.LSTM(100, 150, batch_first=True)
    self.fc = nn.Linear(150, vocab_size)

  def forward(self,x):
    embedded = self.embedding(x)
    intermediate_hidden_states,(final_hidden_state,final_cell_state)=self.lstm(embedded)
    output= self.fc(final_hidden_state.squeeze(0))
    return output

In [181]:
model = LSTMModel(len(vocab))

In [182]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [183]:
model.to(device)

LSTMModel(
  (embedding): Embedding(289, 100)
  (lstm): LSTM(100, 150, batch_first=True)
  (fc): Linear(in_features=150, out_features=289, bias=True)
)

In [184]:
epochs = 50
learning_rate = 0.001

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [185]:
for epoch in range(epochs):
  torch_loss=0

  for batch,labels in dataloader:
    optimizer.zero_grad()
    output= model(batch)
    loss= criterion(output,labels)
    loss.backward()
    optimizer.step()
    total_loss = total_loss + loss.item()

  print(f"Epoch: {epoch + 1}, Loss: {total_loss:.4f}")


Epoch: 1, Loss: 1586.8344
Epoch: 2, Loss: 1733.0054
Epoch: 3, Loss: 1866.5691
Epoch: 4, Loss: 1987.9822
Epoch: 5, Loss: 2098.0618
Epoch: 6, Loss: 2197.4533
Epoch: 7, Loss: 2286.3348
Epoch: 8, Loss: 2365.2763
Epoch: 9, Loss: 2435.4915
Epoch: 10, Loss: 2497.4792
Epoch: 11, Loss: 2552.5955
Epoch: 12, Loss: 2601.4778
Epoch: 13, Loss: 2644.4348
Epoch: 14, Loss: 2681.9108
Epoch: 15, Loss: 2715.1751
Epoch: 16, Loss: 2744.5190
Epoch: 17, Loss: 2770.5894
Epoch: 18, Loss: 2793.4511
Epoch: 19, Loss: 2813.8451
Epoch: 20, Loss: 2832.0689
Epoch: 21, Loss: 2848.4138
Epoch: 22, Loss: 2863.4110
Epoch: 23, Loss: 2876.9769
Epoch: 24, Loss: 2889.4277
Epoch: 25, Loss: 2900.7837
Epoch: 26, Loss: 2911.3009
Epoch: 27, Loss: 2921.1159
Epoch: 28, Loss: 2930.4319
Epoch: 29, Loss: 2939.1212
Epoch: 30, Loss: 2947.2553
Epoch: 31, Loss: 2954.8455
Epoch: 32, Loss: 2962.1704
Epoch: 33, Loss: 2969.2031
Epoch: 34, Loss: 2975.9137
Epoch: 35, Loss: 2982.4020
Epoch: 36, Loss: 2988.5174
Epoch: 37, Loss: 2994.4749
Epoch: 38,

In [186]:
# prediction

def prediction(model, vocab, text):

  # tokenize
  tokenized_text = word_tokenize(text.lower())

  # text -> numerical indices
  numerical_text = text_to_indices(tokenized_text, vocab)

  # padding
  padded_text = torch.tensor([0] * (61 - len(numerical_text)) + numerical_text, dtype=torch.long).unsqueeze(0)

  # send to model
  output = model(padded_text)

  # predicted index
  value, index = torch.max(output, dim=1)

  # merge with text
  return text + " " + list(vocab.keys())[index]



In [187]:
prediction(model, vocab, "The course follows a monthly")

'The course follows a monthly subscription'

In [188]:
import time

num_tokens = 10
input_text = "hi how are"

for i in range(num_tokens):
  output_text = prediction(model, vocab, input_text)
  print(output_text)
  input_text = output_text
  time.sleep(0.5)


hi how are i
hi how are i am
hi how are i am not
hi how are i am not able
hi how are i am not able to
hi how are i am not able to make
hi how are i am not able to make the
hi how are i am not able to make the payment
hi how are i am not able to make the payment on
hi how are i am not able to make the payment on the


In [189]:
dataloader1 = DataLoader(dataset, batch_size=32, shuffle=False)

In [190]:
# Function to calculate accuracy
def calculate_accuracy(model, dataloader, device):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0

    with torch.no_grad():  # No need to compute gradients
        for batch_x, batch_y in dataloader1:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)

            # Get model predictions
            outputs = model(batch_x)

            # Get the predicted word indices
            _, predicted = torch.max(outputs, dim=1)

            # Compare with actual labels
            correct += (predicted == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total * 100
    return accuracy

# Compute accuracy
accuracy = calculate_accuracy(model, dataloader, device)
print(f"Model Accuracy: {accuracy:.2f}%")


Model Accuracy: 95.65%
